# Stochastic Variational Inference for LDA

## Introducción

En el presente notebook se realiza una implementación del algoritmo para topic modelling, Latent Dirichlet Allocation (LDA), en su versión con Stochastic Variational Inference. La implementación trata de apegarse a lo propuesto por Hoffman, Blei y Wang en [Stochastic Variational Inference](http://www.columbia.edu/~jwp2128/Papers/HoffmanBleiWangPaisley2013.pdf).
Otros trabajos que fueron tomados como referencia son:
 * Para un panorama más amplio sobre LDA: [Inference Methods for Latent Dirichlet Allocation](http://times.cs.uiuc.edu/course/598f16/notes/lda-survey.pdf);
 * Para una explicación más detallada sobre la Mean-Field Variational Family: [Variational Inference: A Review for Statisticians](https://arxiv.org/pdf/1601.00670.pdf)
 
Adicionalmente, la idea de implementar LDA surgió de un intento previo de querer implementar la propuesta de Wang y Blei, en [Collaborative Topic Modeling for Recommending Scientific Articles](http://www.cs.columbia.edu/~blei/papers/WangBlei2011.pdf), para Sistemas de Recomendación. De hecho el dataset utilizado aqui es el mismo que usan ellos. No descarto implementar este paper como siguiente paso.

## Marco teórico

Asumiendo K tópicos, y D documentos, cada uno con N palabras (por simplicidad asumo documentos de igual longitud) pertenecientes a un vocabulario de tamaño V; el modelo generativo es el siguiente:

1. Generamos tópicos $\beta_k$ ~ Dirichlet($\eta,\dots,\eta$) para cada $k\in \{0,\dots,K-1\}$
2. Para cada documento $d \in \{0,\dots, D-1\}$:
    3. Generamos $\theta_d$ ~ Dirichlet($\alpha,\dots,\alpha$)
    4. Para cada palabra $w \in \{0,\dots, N-1\}$:
        5. Generamos la asignación de tema $z_{dn}$ ~ Multinomial($\theta_d$)
        6. Generamos la palabra $w_{dn}$ ~ Multinomial($\beta_{z_{dn}}$)

Los parámetros involucrados son:
* $\beta_k$, el tópico $k$. Un tópico consiste básicamente en un vector de probabilidad de longitud V, que modela la distribución de las distintas palabras en el vocabulario para ese tema
* $\theta_d$, la proporción de los tópicos para el documento $d$. Nuevamente es un vector de probabilidades, que en este caso apunta a modelar la participación parcial de los distintos temas para un mismo documento. El hecho de que se permita que un documento pertenezca a un mixture de tópicos en lugar de uno solo, es una de las principales virtudes de LDA. __Esta es la principal variable sobre la que nos interesa poder hacer inferencia__.
* $z_{dn}$, es la asignación del tópico de la palabra $n$ en el domunento $d$
* $w_{dn}$, es la palabra $n$ del documento $d$. Pertenece al tópico $z_{dn}$

## Dataset

El dataset fue descargado de http://www.cs.cmu.edu/~chongw/data/citeulike/. El mismo contiene información del sitio http://www.citeulike.org/, que le permite a investigadores armar sus bibliotecas de papers y recibir recomendaciones en base a las mismas. En particular yo no uso la información referente a los usuarios y sus bibliotecas, si no que únicamente me centro en la data sobre artículos.
Los dos archivos que uso son:
* mult.dat, que contiene los ids de las palabras más relevantes de cada artículos, y sus conteos. En total son 16980 artículos (documentos).
* vocab.dat, el mapeo de los ids a la palabra específica. El tamaño del vocabulario usado es de 8000 palabras.

La extracción del vocabulario, y demás preprocesamientos de la data son los que se explican en la sección 4 de [Wang and Blei (2011)](http://www.cs.columbia.edu/~blei/papers/WangBlei2011.pdf).

Adicionalmente, en esta primera iteración del algoritmo, pido que todos los documentos tengan la misma cantidad de palabras. Eso me lleva a realizar un procesamiento extra, en el cual descarto documentos con menos de 40 palabras, quedandome con 14836. A su vez, para estos artículos, solo conservo las primeras 40 palabras, ordenadas de mayor a menor ocurrencia en el documento.

A continuación, importo algunas bibliotecas que voy a usar, y realizo la carga del dataset. Por cuestiones de reproducibilidad, seteo una semilla fija.

In [1]:
from scipy.stats import dirichlet, multinomial, expon
from scipy.special import digamma
from collections import defaultdict
from math import pow
import numpy as np
import copy

from online_lda import OnlineLDA
from mini_batch_lda import MiniBatchLDA
from helpers import *

In [2]:
def load_dataset():
    words_per_doc = defaultdict(list)
    # Parse line by line the documents data's file
    with open('data/mult.dat') as keywords_f:
        words_and_counts_by_doc = []
        for l in keywords_f:
            data = l.split() # First element is the number of distinct words, the rest are the word_ids with its counters 
            words_and_counts_by_doc.append(data[1:])
        for d, words_and_counts in enumerate(words_and_counts_by_doc):
            for w_c in words_and_counts:
                word, cnt = w_c.split(':')
                for _ in xrange(int(cnt)):    
                    words_per_doc[d].append(int(word))
            # Resampling the data to always have the same number of words per document
            np.random.seed(42)
            words_per_doc[d] = np.random.choice(words_per_doc[d], size=60)
    
    vocab = {}
    with open('data/vocab.dat') as vocab_f:
        for v, word in enumerate(vocab_f):
            vocab[v] = word.rstrip()
    
    return words_per_doc, vocab
            

In [3]:
words_by_doc, vocab = load_dataset()

## LDA

Primero me defino algunas funciones auxiliares que me van a servir luego.

In [4]:
from IPython.display import display, Markdown
def print_md(string, bold=False):
    if bold:
        string = '**' + string + '**'
    display(Markdown(string))

La clase _LatentDirichletAllocation_ tiene como método principal _fit_, que es el que se encarga de aplicar el algoritmo propuesto por Hoffman et al. (2013). La notación elegida para los parámetros sigue fielmente la usada en la sección 3 de dicho paper. 

La imagen siguiente muestra el pseudocódigo del algoritmo en cuestión.

![title](img/SVI-pseudocode.png)

In [5]:
D = len(words_by_doc)  # number of documents
N = 60 # words per doc
K = 60  # number of topics
V = len(vocab)  # vocabulary size

# For alpha and eta values I follow recommendation from Griffiths and Steyvers (2004)
lda = OnlineLDA(K, D, N, V, 50.0/K, 0.1, words_by_doc)
lda.fit()

Iteracion: 0
Documento: 5770
Ro: 0.0334929207043
Iteracion: 1
Documento: 11239
Ro: 0.0333109641294
Iteracion: 2
Documento: 11493
Ro: 0.0331313897449
Iteracion: 3
Documento: 661
Ro: 0.0329541483957
Iteracion: 4
Documento: 12864
Ro: 0.032779192306
Iteracion: 5
Documento: 7180
Ro: 0.0326064750308
Iteracion: 6
Documento: 363
Ro: 0.0324359514088
Iteracion: 7
Documento: 4779
Ro: 0.0322675775179
Iteracion: 8
Documento: 5343
Ro: 0.0321013106322
Iteracion: 9
Documento: 1663
Ro: 0.0319371091812
Iteracion: 10
Documento: 6505
Ro: 0.03177493271
Iteracion: 11
Documento: 14368
Ro: 0.0316147418418
Iteracion: 12
Documento: 9360
Ro: 0.0314564982415
Iteracion: 13
Documento: 13164
Ro: 0.0313001645812
Iteracion: 14
Documento: 11476
Ro: 0.0311457045065
Iteracion: 15
Documento: 7843
Ro: 0.0309930826047
Iteracion: 16
Documento: 4408
Ro: 0.0308422643739
Iteracion: 17
Documento: 11600
Ro: 0.0306932161937
Iteracion: 18
Documento: 4302
Ro: 0.0305459052965
Iteracion: 19
Documento: 6823
Ro: 0.0304002997402
Iteracio

In [6]:
for i, lamb_k in enumerate(lda.lamb):
    print_md('Topic {}'.format(i), bold=True)
    np.random.seed(42)
    words_distribution = np.random.dirichlet(lamb_k)
    #print words_distribution.argsort()[::-1][:10]
    #words_distribution.sort()
    #print words_distribution[::-1][:10]
    print_words(words_distribution.argsort()[::-1][:20], vocab)

**Topic 0**

72, techniques
118, text
642, hierarchical
377, integration
385, reference
279, basis
349, traditional
619, visualization
566, tags
574, tagging
478, edition
612, expected
766, tag
951, scales
910, easily
1366, systematically
721, end
1092, reading
1125, index
1132, reasoning


**Topic 1**

99, transcription
163, mechanisms
329, shows
432, experience
460, address
669, relations
585, degree
578, early
459, library
586, categories
1143, optical
864, affect
1178, supporting
734, libraries
1427, realtime
1064, relation
1752, explanation
2596, classroom
863, idea
1165, remain


**Topic 2**

...

206, communities
345, engineering
425, action
276, authors
957, package
645, generate
636, take
1113, estimated
667, theories
832, motor
1243, ranking
1183, understood
501, able
872, conceptual
1252, international
828, measured
845, nonlinear
869, correlations
788, analyzing
787, total


**Topic 58**

259, variety
232, type
200, quantum
181, examples
383, likely
155, target
391, resource
936, last
330, conservation
744, classical
592, personal
877, formation
958, scheme
954, microrna
833, word
1000, perception
1540, expansion
1737, scenarios
1344, differ
1704, viewed


**Topic 59**

132, terms
146, possible
135, application
156, factors
1282, explains
380, association
473, provided
404, genomewide
396, ways
540, robust
1264, cognition
486, map
1909, examining
618, author
944, freely
663, diseases
881, appear
1042, writing
1184, subset
674, products


In [7]:
D = len(words_by_doc)  # number of documents
N = 60 # words per doc
K = 60  # number of topics
V = len(vocab)  # vocabulary size
minibatch_size = 32

# For alpha and eta values I follow suggestion from Griffiths and Steyvers (2004)
lda = MiniBatchLDA(K, D, N, V, 50.0/K, 0.1, words_by_doc, minibatch_size)
lda.fit()

Iteracion: 0
Documentos: [ 5770 12459  9969  6671 10870 13254 16844  2418 13242 15500   761  3410
  6391 13336 11433 11602  1535 13834  8740  2025  5971  8832  5380 15532
  4029 11998  6332 14655 11041  8740  2748  2060]
Ro: 0.0334929207043
Iteracion: 1
Documentos: [ 7156  1830 14457 13098  6566   397  7317 13271  6759  9351 14493 16739
  4445  2667   844 15046 11211 14846  4267  5498 15313 10708 10266 11143
 15683  5904  5191  5391 15200 16471  2321 12164]
Ro: 0.0333109641294
Iteracion: 2
Documentos: [ 4862 11376 16019  8746 11012 14519  2815  8677 13285 13336  8341 12805
  5307  1455 13226 13655 15638  9548 12009 10438 12804  2581  7766 12724
 12335 15085  3575  3469  6148 13103 11208  5647]
Ro: 0.0331313897449
Iteracion: 3
Documentos: [ 9889  5600  2013  8645 15581  4203 13504  7497  5687 14826  9410 16863
 13965  9624  4634 13494 13013 14370 10219  5723  1508  5049   707 12477
 14617  2483  7613  7214  9638  8937 12783  7451]
Ro: 0.0329541483957
Iteracion: 4
Documentos: [14580  411

In [8]:
for i, lamb_k in enumerate(lda.lamb):
    print_md('Topic {}'.format(i), bold=True)
    np.random.seed(42)
    words_distribution = np.random.dirichlet(lamb_k)
    #print words_distribution.argsort()[::-1][:10]
    #words_distribution.sort()
    #print words_distribution[::-1][:10]
    print_words(words_distribution.argsort()[::-1][:20], vocab)

**Topic 0**

149, associated
210, image
642, hierarchical
382, linear
251, code
228, allows
448, input
299, pattern
619, visualization
373, images
286, recognition
451, responses
1538, arbitrary
612, expected
774, conclusions
719, specificity
659, representations
709, testing
625, connectivity
421, diverse


**Topic 1**

24, method
163, mechanisms
371, essential
669, relations
327, called
699, explain
682, popular
1178, supporting
726, adaptation
916, cultural
976, reliable
891, conventional
863, idea
871, likelihood
938, readers
1361, phenomenon
1883, slow
745, synaptic
1780, reasons
2561, illustrates


**Topic 2**

...

98, changes
276, authors
337, probability
230, quality
344, empirical
389, generated
282, cellular
627, projects
983, wikipedia
702, tests
673, combined
894, mean
1213, reconstruction
1019, suggested
856, collaboration
1171, detecting
1186, agent
957, package
1075, objective
1349, traffic


**Topic 58**

20, social
46, provide
202, technologies
189, major
248, relationships
184, article
199, groups
369, implementation
242, group
300, interest
592, personal
366, develop
1027, strongly
493, primary
789, define
936, last
919, collective
1539, focuses
940, notion
1087, matter


**Topic 59**

161, effects
348, findings
540, robust
374, measures
618, author
1029, conclude
1282, explains
695, term
881, appear
972, explicit
811, suggesting
396, ways
1369, locations
1057, commonly
1269, scoring
2179, teachers
1909, examining
1208, modification
3331, circuitry
1089, reflect
